In [6]:
import tvm
from tvm import te

N, M, L = 1024, 1024, 1024
A = te.placeholder((N, L), name='A')
B = te.placeholder((L, M), name='B')
k = te.reduce_axis((0, L), 'k')
C = te.compute((N, M), lambda i, j: te.sum(A[i, k] * B[k, j], axis=k), name='C')

s = te.create_schedule(C.op)
block_x = te.thread_axis("blockIdx.x")
thread_x = te.thread_axis("threadIdx.x")
block_y = te.thread_axis("blockIdx.y")
thread_y = te.thread_axis("threadIdx.y")

# 选择一个合适的块和线程块大小来分解C的计算
bx, tx = s[C].split(C.op.axis[0], factor=32)
by, ty = s[C].split(C.op.axis[1], factor=32)

s[C].bind(bx, block_x)
s[C].bind(tx, thread_x)
s[C].bind(by, block_y)
s[C].bind(ty, thread_y)

mod = tvm.build(s, [A, B, C], 'cpu')


ValueError: Traceback (most recent call last):
  File "/Users/xiangyy/tvm/src/target/target.cc", line 580
ValueError: Target kind "CPU" is not defined. Target creation from string failed: CPU